In this notebook, different statistics of the detected ramp planes will be analyzed and visualized.<br>
Also a score will be calculated to determine how well the algorithm works.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
# Load offline ramp detection algorithm
from lidar_ramp_offline import VisualDetection
# Load methods to extract data from rosbags
from getData import unpack_bag, synchronize_topics
# Load custom plots
from plot_library_lidar import PlotLib

pio.templates.default = "plotly_dark"
# %load_ext blackcellmagic


#### **The following values need to be adjusted for each rosbag**


In [ ]:
#! Select a bag
BAG_NUM = 1

BAG_INFO = [
    {
        "bag_name": "u_c2s_half_odom_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[20.3, 33], [-0.9, 2.8]],
    },
    {
        "bag_name": "u_c2s_half_odom_stereo_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[23.8, 36], [-3.3, 0.5]],
    },
    {
        "bag_name": "u_c2s_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[45, 58], [-1.9, 1.8]],
    },
    {
        "bag_name": "u_c2s_stop_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[38, 51], [-1.5, 2.2]],
    },
    {
        "bag_name": "u_d2e_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[32.5, 44], [2, 5.5]],
    },
    {
        "bag_name": "u_s2c_half_odom_hdl.bag",
        "ramp_type": "uc",
        "xy_range": [[42, 56], [-2.2, 2]],
    },
    {
        "bag_name": "u_s2c2d_part1_hdl.bag",
        "ramp_type": "uc",
        "xy_range": [[47.3, 62], [-2.8, 1.5]],
    },
    {
        "bag_name": "u_s2c2d_part2_hdl.bag",
        "ramp_type": "us",
        "xy_range": [[47, 58.8], [36.5, 40.5]],
    },
]

# Rosbag path
BAG_PATH = "/home/user/rosbags/final/slam/" + BAG_INFO[BAG_NUM]["bag_name"]

# ROS topics
ODOM_TOPIC = "/odom"
LIDAR_TOPIC = "/velodyne_points"

# Ground truth coordinates of ramp (measured by using globalmap points)
X_RANGE, Y_RANGE = BAG_INFO[BAG_NUM]["xy_range"]

Run algorithm with rosbag data.<br>
Store many different calculated metrics generated by the algorithm in a pandas data frame (to allow a faster plotting).


In [ ]:
class GetScore():
    """Run lidar ramp detection on rosbag and calculate some scores"""

    def __init__(self, bag_path, x_range, y_range):
        self.bag_path = bag_path
        self.x_range = x_range
        self.y_range = y_range

    @staticmethod
    def extract_data(bag_path):
        """Get odometer and lidar data from rosbag"""
        # Odometer from hdl_slam
        pose_async, t_pose = unpack_bag(bag_path, ODOM_TOPIC, "hdl_odom")
        # Velodyne point cloud
        lidar_async, t_lidar = unpack_bag(bag_path, LIDAR_TOPIC)

        # Synchronize both topics (odom usually is behind lidar)
        pose, lidar = synchronize_topics(pose_async, t_pose, lidar_async, t_lidar)
        return lidar, pose

    def run_the_algorithm(self, lidar, pose):
        """Perform ramp detection on data"""
        # Create instance of class (using standard parameters):
        vd = VisualDetection(self.x_range[0])
        # Lists to fill, will contain entry for each frame
        planes = []
        ramp_stats = []
        true_dists = []
        all_points = []
        for i, _ in enumerate(lidar):
            plane_points, data, pc_whole = vd.spin(lidar[i], pose[i])
            # Point coordinates of points detected as ramp
            planes.append(plane_points)
            # Point coordinates of all points (for visualization)
            all_points.append(pc_whole)
            # Ramp stats [angle, width, dist, true_dist]
            ramp_stats.append(data)
            # True distance to start of ramp
            # (necessary because ramp_stats empty for frames where no ramp)
            true_dist = self.x_range[0] - pose[i].position.x
            true_dists.append(true_dist)
        return planes, ramp_stats, true_dists, all_points

    @staticmethod
    def get_ramp_frames(planes, plane_stats):
        """Removes all samples, where no ramp has been detected"""
        # Remove samples where no ramp has been detected
        ramp_arrays = [x for x in planes if not isinstance(x, list)]
        ramp_stats = [x for x in plane_stats if x]
        # Get indices where ramp has been detected
        ramp_indices = [i for i, v in enumerate(planes) if not isinstance(v, list)]
        return ramp_arrays, ramp_stats, ramp_indices

    @staticmethod
    def convert_to_df(ramp_arrays):
        """Convert array of as ramp detected points into a dataframe
        and add sample and point index. Each row is one point"""
        # Convert list to dictionary
        dic = []
        for i, arr in enumerate(ramp_arrays):
            for j, point in enumerate(arr):
                dic.append(
                    {
                        "sampleIdx": i,
                        "pointIdx": j,
                        "x": point[0],
                        "y": point[1],
                        "z": point[2],
                    }
                )
        # And finally to pandas data frame
        ramp_points_df = pd.DataFrame(dic)
        # Reorder columns
        ramp_points_df = ramp_points_df[["sampleIdx", "pointIdx", "x", "y", "z"]]
        return ramp_points_df

    def ground_truth_check(self, ramp_points_df, ramp_stats, only_before_ramp=True):
        """Calculate percentage of as ramp identified points, which
        do really inside the ramp region, and return them together
        with estimated angle, width, distance and true distance in a df"""
        # Check if a point lies within ramp region
        lies_inside = []
        for i, x in enumerate(ramp_points_df["x"]):
            if (
                self.x_range[0] < x < self.x_range[1]
                and self.y_range[0] < ramp_points_df["y"][i] < self.y_range[1]
            ):
                # True if x and y coordinate inside region
                lies_inside.append(True)
            else:
                lies_inside.append(False)
        # Add column (bool: if point lies in region) to data frame
        ramp_points_df["inlier"] = lies_inside

        # Calculate how many points of each sample lie in ramp region
        true_inliers = []
        samples_num = ramp_points_df.sampleIdx.max() + 1
        for i in range(samples_num):
            # Bool list of inliers and outliers of sample
            bool_lst = ramp_points_df[ramp_points_df["sampleIdx"] == i]["inlier"]
            # Percentage of inliers of sample
            true_inlier_perc = sum(bool_lst) / float(len(bool_lst))
            true_inliers.append(true_inlier_perc)

        # New dataframe with stats for each frame
        # Structure reminder of ramp_stats: [angle, width, dist, true_dist]
        dic = []
        for i in range(samples_num):
            dic.append(
                {
                    "sampleIdx": i,
                    "TrueInliers": true_inliers[i],
                    "Angle": ramp_stats[i][0],
                    "Width": ramp_stats[i][1],
                    "Dist": ramp_stats[i][2],
                    "TrueDist": ramp_stats[i][3],
                }
            )
        # Convert dictionary to dataframe
        df_stats = pd.DataFrame(dic)
        # Reorder columns
        df_stats = df_stats[["sampleIdx", "TrueInliers", "Angle", "Width", "Dist", "TrueDist"]]
        # Remove all estimations from when after the ramp has been entered
        if only_before_ramp:
            df_stats = df_stats[df_stats["TrueDist"] > 0]
        return df_stats

    def calc_score(self, df_stats, true_dists, min_dist, max_dist, step_dist):
        """Calculates some scores to determine performance"""
        scores = []
        for _, min_d in enumerate(range(min_dist, max_dist, step_dist)):
            max_d = min_d + step_dist
            # Samples recorded in the given distance range
            true_dists = np.asarray(true_dists)
            # Assuming ramp is visible in every frame --> num of frames == expected detections
            frames = len(true_dists[(min_d < true_dists) & (true_dists < max_d)])
            # Actual detections only counts if at least 50% of points lie inside ramp region
            detections_tp = len(
                df_stats[
                    (min_d < df_stats["TrueDist"])
                    & (df_stats["TrueDist"] < max_d)
                    & (df_stats["TrueInliers"] > 0.5)
                ]
            )
            # Ramp has been detected, but less than 50% of points lie in ramp region
            detections_fp = len(
                df_stats[
                    (min_d < df_stats["TrueDist"])
                    & (df_stats["TrueDist"] < max_d)
                    & (df_stats["TrueInliers"] < 0.5)
                ]
            )
            # Calculate ratio
            try:
                true_positives = float(detections_tp) / frames * 100
                false_positives = float(detections_fp) / frames * 100
            except ZeroDivisionError:
                true_positives = np.NaN
                false_positives = np.NaN
            scores.append((min_d, max_d, frames, true_positives, false_positives))
        return scores
    
    def vis_score(self, scores):
        """Needed for bar plot"""
        # Only use distance to ramp, # frames and # detections columns
        detection = np.asarray(scores)[:, [0, 2, 3]]
        # Convert to pandas DataFrame
        df_eval = pd.DataFrame(detection)
        # Rename columns
        df_eval.columns = ["distToRamp", "expectedDetections", "actualDetections"]
        return df_eval

In [ ]:
gs = GetScore(BAG_PATH, X_RANGE, Y_RANGE)
lidar, pose = gs.extract_data(BAG_PATH)
planes, plane_stats, true_dists, all_points = gs.run_the_algorithm(lidar, pose)
ramp_arrays, ramp_stats, ramp_indices = gs.get_ramp_frames(planes, plane_stats)
ramp_points_df = gs.convert_to_df(ramp_arrays)
df_stats = gs.ground_truth_check(ramp_points_df, ramp_stats)
scores = gs.calc_score(df_stats, true_dists, min_dist=0, max_dist=30, step_dist=1)
df_eval = gs.vis_score(scores)

# Plots


In [ ]:
# Load my plot library
plt = PlotLib(ramp_points_df, df_stats, df_eval, X_RANGE, Y_RANGE, all_points, ramp_indices)

**How was this plot created:**

- Calculate distance to start of ramp using odometry data from hdl_slam
- Calculate how many lidar samples were recorded during a 1m drive (e.g. car took 0.5s for 1m $\rightarrow$ lidar took 5 samples (because $f_\text{Lidar}=$ 10Hz))
- Calculate how many samples of them were actually identified as a ramp by the algorithm
- Represent both values in a bar plot
- Value at bar represents the lower range (e.g. bar at 15 means 16m to 15m)

**What to take from this plot:**<br>

- At which distance to the ramp is the lidar most reliable?<br>
- $\rightarrow$ Range between 10m to 4m seems to be the most reliable


In [ ]:
plt.bar_plot()

**How was this plot created:**

- Every time a ramp is detected certain metrics such as angle, width, distance are estimated by the algorithm
- Compare predictions to the true values, true values were measured by hand
- Calculate some stats like standard deviation or average error and add them
- Represent values as a line plot

**What to take from this plot:**

- (Lars wanted it)
- Distance to ramp:
  - Distance is estimated quite well up to about 3m before the ramp
  - There does seem to be a slight offset (manual measurement might have been wrong)
- Angle:
  - Quite a bit of variance at far and close distance, but very small deviation in the range 9m to 3m
  - Because ramp does not have a constant angle, angle estimation of 4° etc. at the beginning of the ramp might be correct
- Width:
  - Mhm...


In [ ]:
plt.distance_estimation(False)
plt.angle_estimation(7).show()
plt.width_estimation(2.9).show()

Making the plots ready for latex export

In [ ]:
pio.templates.default = "plotly_white"

fig = plt.distance_estimation(False)
fig2 = plt.angle_estimation(7, False)
fig3 = plt.width_estimation(2.9, False)
figs = [fig, fig2, fig3]
    
for fig in figs:
    fig.update_layout(
        width=600,
        height=300,
        font_family="Serif",
        font_size=14,
        font_color="black",
        margin_l=5,
        margin_t=5,
        margin_b=5,
        margin_r=5,
        title="",
        legend=dict(x=0.7, y=1, traceorder="normal", bordercolor="Gray", borderwidth=1),
    )
    fig.show()
    # fig.write_image("dist.pdf")



**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inliers are stored
- But often times not all points in ramp region are detected
- Display points detected by algorithm in one color and all the other points recorded by the lidar in another color
- Add an area of the true ramp region (measured by hand)
- Calculate percentage of inliers how lie in ramp region
- Do this for every sample which was detected as ramp
- Add a slider (just to show off)

**What to take from this plot:**

- (Plotly is cool)
- Only very few "lines" are thrown on the ground / on to the ramp
- Distances of over 2m between two lines are common (especially at higher distances)


In [ ]:
plt.animation_only_detections()

**How was this plot created:**

- Similar to previous plot
- For every recorded lidar sample all points are displayed
- Add an area of the true ramp region (measured by hand)

**What to take from this plot:**

- Just gives an idea of what the lidar sees, might be useful to optimize mount pitch angle
- Also might give information, about why some samples are not recognized as ramp, eventhough they should be


In [ ]:
fig = go.Figure()
# Add traces, one for each slider step
for step in range(len(all_points)):
    fig.add_trace(
        go.Scatter(
            visible=False,
            mode="markers",
            x=all_points[step][:, 0],
            y=all_points[step][:, 1],
            name="All lidar points",
        )
    )
fig.data[0].visible = True
fig.add_shape(
    type="rect",
    x0=X_RANGE[0],
    x1=X_RANGE[1],
    y0=Y_RANGE[0],
    y1=Y_RANGE[1],
    fillcolor="red",
    opacity=0.2,
)
# Set static axes limits
fig.update_xaxes(range=x_fixed)
fig.update_yaxes(range=y_fixed)

# Create and add slider
steps = []
for i in range(len(fig.data)):
    ramp_det = "A" if i in ramp_indices else "No"
    step = dict(
        method="update",
        args=[
            {"visible": [False] * len(fig.data)},
            {
                "title": "{:.2f}m infront of ramp. {} ramp has been detected".format(
                    #   true_inliers[i]*100, df_stats.iloc[i,-1])}],  # layout attribute
                    true_dist[i],
                    ramp_det,
                )
            },
        ],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(active=0, currentvalue={
                "prefix": "Deteced plane: "}, steps=steps)]
fig.update_layout(
    sliders=sliders,
    xaxis_title="Global x coor [m]",
    yaxis_title="Global y coor [x]",
    showlegend=True,
)
fig.show()


**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inlier points are saved
- Put all point coordinates of every detection together
- Add an area of the true ramp region (measured by hand)
- Display as a scatter plot and also as a heatmap

**What to take from this plot:**

- Shows which points are most commonly detected
- Majority of points are inside ramp region
- ?


In [ ]:
plt.all_detected_points().show()
plt.all_detected_points(show_as_heatmap=True).show()